## Parte 1 :   Importando Bibliotecas que serão usadas

In [307]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import heapq as hp
import time
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



## Lendo arquivo csv
## Eliminando caracteres com tamanho 2 ou menor e deixando todos em caixa baixa




In [308]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
palavras = []


texto.head()

0    [juíza, federal, ivani, silva, luz, brasília, ...
1    [menos, horas, depois, juíza, federal, ivani, ...
2    [quando, determinou, que, março, uma, estratég...
3    [quando, era, pequeno, luisa, gabriel, preocup...
4    [desde, manhã, última, segunda, feira, sucesso...
Name: text, dtype: object

## Eliminando as stopwords : Usando a biblioteca nltk
## Algoritmo Build Index : Usada para uma indexação simples


In [312]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 

def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      par = (numero_da_lista, ocorrencia)
      if palavra not in dic_index:
        dic_index[palavra] = []
      
      if par not in dic_index[palavra]:
         dic_index[palavra].append(par)
  return dic_index
                
index = build_index(texto)
tabela = {'palavra' : list(index.keys()),
       'doc|ocorrencia' : list(index.values()) }
df = pd.DataFrame(tabela)
df.head()



,palavra,doc|ocorrencia
0,juíza,"[(1, 2), (2, 1)]"
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ..."
2,ivani,"[(1, 1), (2, 1)]"
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73..."
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78..."


# Parte 2 : Implementação do processamento de consulta documento-por-vez e termo-por-vez



## Documento Por Vez : Calcula escores completos para documentos processando todas as listas de termos, um documento por vez

In [313]:
def document_por_vez(Q, I, k):
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for document in range(1, len(texto)+1):
        sd = 0
        for l in L:
            for i in l:
                if (i[0] == document):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(R, (sd, document))
    return heapq.nlargest(k, R)
document_por_vez("onu", index, 10)

[(2, 101),
 (2, 64),
 (2, 29),
 (1, 249),
 (1, 185),
 (1, 97),
 (1, 70),
 (1, 42),
 (1, 8),
 (1, 3)]

## Termo Por Vez: Acumula escores para documentos processando listas de termos, um termo por vez

In [314]:
def termo_por_vez(Q, I, k):
    A = {}
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for l in L:
        for post in l:
            d = post[0]
            freq = post[1]
            if (d in A.keys()):
                A[d] += freq
            else:
                A[d] = freq
    for (d, ad) in A.items():
        sd = ad
        heapq.heappush(R, (sd, d))
    return heapq.nlargest(k, R)

termo_por_vez("onu", index, 10)

[(2, 101),
 (2, 64),
 (2, 29),
 (1, 249),
 (1, 185),
 (1, 97),
 (1, 70),
 (1, 42),
 (1, 8),
 (1, 3)]

##Definindo consultas de um termo 

In [0]:
queries = ["política", "presidente", "bolsonaro", "maduro", "onu"]

## Executa 5 consultas em cada algoritmo retornando os top-10 

In [0]:
  # Documento
  result_doc = []
  time_doc = []
  # Termo
  result_term = []
  time_term = []
  k = 10
  for query in queries:
    # Documento por vez
    inicio_doc = time.time()
    result_doc.append(document_por_vez(query, index, k))
    fim_doc = time.time()
    time_doc.append(fim_doc - inicio_doc)
    # Termo Por Vez
    inicio_term = time.time()
    result_term.append(termo_por_vez(query, index, k))
    fim_term = time.time()
    time_term.append(fim_term - inicio_term)
  # criação da tabela
  tabela = pd.DataFrame()
  tabela['query'] =  queries
  tabela['documento_por_vez'] = result_doc
  tabela['termo_por_vez'] = result_term
  tabela['comparando'] = tabela.documento_por_vez == tabela.termo_por_vez

## Resultado Top-10 documentos

In [326]:
tabela
# tabela.to_csv('queries.csv')

,query,documento_por_vez,termo_por_vez,comparando
0,política,"[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...","[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...",True
1,presidente,"[(21, 63), (16, 166), (12, 151), (11, 216), (1...","[(21, 63), (16, 166), (12, 151), (11, 216), (1...",True
2,bolsonaro,"[(46, 151), (37, 166), (35, 207), (26, 19), (1...","[(46, 151), (37, 166), (35, 207), (26, 19), (1...",True
3,maduro,"[(11, 150), (7, 99), (6, 65), (5, 88), (4, 247...","[(11, 150), (7, 99), (6, 65), (5, 88), (4, 247...",True
4,onu,"[(2, 101), (2, 64), (2, 29), (1, 249), (1, 185...","[(2, 101), (2, 64), (2, 29), (1, 249), (1, 185...",True


##Ao compararmos as consultas  podemos perceber que temos o mesmo resultado, indicando que o algoritimos estão concordando entre si.

## Comparando os tempos médios de execução

In [319]:
tabela_comparar = {
    'tempo médio documento': time_doc,
    'tempo médio termo' : time_term
}

tabela_df = pd.DataFrame(tabela_comparar)
tabela_df

,tempo médio documento,tempo médio termo
0,0.001326,0.000110
1,0.001811,0.000145
2,0.001055,0.000090
3,0.000245,0.000019
4,0.000225,0.000012


##Chegando a conclusão que o tempo medio do algoritmo termo-por-vez  é menor que documento-por-vez 

#Parte 3 :  Consulta conjuntiva

## Implemente uma das versões de consulta conjuntiva (AND) 

In [320]:
def termConj(Q, invertedIndex, k):
  L = []
  R = []

  for term in Q.split():
    if term in invertedIndex.keys():
        L.append(invertedIndex[term])
  
  termos = [item for sublist in L for item in sublist]
  termos.sort()
  for i in range(len(termos)):
    score = 0
    d = termos.pop()
    aux = 1
    for index in termos:
      if index[0] == d[0]:
        score += index[1] + d[1]
        aux += 1
    if score != 0 and aux == len(L):
      hp.heappush(R, (d[0], score))
  

  return hp.nlargest(k, R)

termConj('com o escritório em Jerusalém  o presidente brasileiro evita por ora seguir os passos dos', index, 5)

[(8, 41)]

## Criando novas consultas

In [323]:
queries = ['livros de historia','crise na bolsa', 
           'família biológica mesmo quando convalescia e recebia ameaças', 
           'Com o escritório em Jerusalém  o presidente brasileiro evita por ora seguir os passos dos', 
           'juíza federal ivani silva', 
           'maduro ditadura']

resultado = []
  
for consulta in queries:
  resultado.append(termConj(consulta, index, 10))

consulta = {
    'Query': queries,
    'Resultado("AND")' : resultado
}

df = pd.DataFrame(consulta)
df
# df.to_csv('parte3.csv')

  

,Query,"Resultado(""AND"")"
0,livros de historia,"[(208, 2), (3, 4)]"
1,crise na bolsa,"[(215, 4), (143, 6), (138, 7), (84, 2), (69, 8)]"
2,família biológica mesmo quando convalescia e r...,"[(6, 42)]"
3,Com o escritório em Jerusalém o presidente br...,"[(8, 41)]"
4,juíza federal ivani silva,"[(2, 9), (1, 14)]"
5,maduro ditadura,"[(63, 3)]"


## Quando  utilizo a função termConj com uma pequena frase os documentos que tiverem essa parte serão retornados . E quando uso Frases grandes que só estar em um documento ele retorna apenas um documento. Isso deixa claro que a função está correta

In [324]:
# Exemplo frase grande

termConj('As seções de treinamento foram registradas pelo cineasta na Marine Corps Base 29 Palms  na Califórnia  em 2009. Esse trabalho também aborda a combinação entre computação gráfica e  em programas utilizados', index, 5)

[(12, 56)]

In [325]:
# Exemplo frase pequena
termConj('presidente brasileiro', index, 5)

[(238, 5), (237, 3), (231, 4), (230, 3), (225, 6)]